## CONNECTION TO Db2 DATABASE ON CLOUD USING PYTHON ##

based on the Coursea course Databases and SQL for Data Science with Python

In [2]:
'''first we need to install sqlalchemy (a python SQL toolkit) and import ibm_db API to access and manipulate data on IBM server database using functions to 
issue SQL statements, fetching rows from result sets etc.'''

!pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
#Ensure we don't load_ext with sqlalchemy>=1.4 (incompadible)
!pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
!pip install ipython-sql


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 KB 3.5 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 KB 1.9 MB/s eta 0:00:0000:01
  Created wheel for ibm_db: filename=ibm_db-3.1.0-cp38-cp38-macosx_10_15_x86_64.whl size=32715576 sha256=12fb9a4e8caa6929ec2347f8bfd3d871c4ad2e539ac41c7b370190e6223fe2d9
  Stored in directory: /Users/darja/Library/Caches/pip/wheels/43/71/0a/3065745bc1dc4cafbdcb09e2557082609f7689b16108b670df
  Created wheel for ibm_db_sa: filename=ibm_db_sa-0.3.3-py3-none-any.whl size=27409 sha256=f6d8cb6e9444f0d99646b6c947765d4aaa54dce02266415644f39d9bf77a7cf6
  Stored in directory: /Users/darja/Library/Caches/pip/wheels/ca/a9/b

In [3]:
import ibm_db

#### Set up credentials for connection to the database ####

In [4]:
#Replace the placeholder values with your actual Db2 hostname, username, and password:
dsn_hostname = "21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud" # e.g.: "54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
dsn_uid = "vsq88098"        # e.g. "vsq88098"
dsn_pwd = "xxxxx"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "31864"                # e.g. "32733" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_security = "SSL"              #i.e. "SSL"

#### Create the connection to the database ####

Ibm_db API uses the IBM Data Server Driver for ODBC and CLI APIs to connect to IBM DB2 and Informix.

In [5]:
#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

#print the connection string to check correct values are specified
print(dsn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud;PORT=31864;PROTOCOL=TCPIP;UID=vsq88098;PWD=DAlSNlkLnNxTgqSc;SECURITY=SSL;


In [6]:
#Create database connection

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )


Connected to database:  BLUDB as user:  vsq88098 on host:  21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud


In [7]:
#Retrieve Metadata for the Database Server
server = ibm_db.server_info(conn)

print ("DBMS_NAME: ", server.DBMS_NAME)
print ("DBMS_VER:  ", server.DBMS_VER)
print ("DB_NAME:   ", server.DB_NAME)

DBMS_NAME:  DB2/LINUXX8664
DBMS_VER:   11.05.0700
DB_NAME:    BLUDB


In [10]:
#Retrieve Metadata for the Database Client / Driver
client = ibm_db.client_info(conn)

print ("DRIVER_NAME:          ", client.DRIVER_NAME) 
print ("DRIVER_VER:           ", client.DRIVER_VER)
print ("DATA_SOURCE_NAME:     ", client.DATA_SOURCE_NAME)
print ("DRIVER_ODBC_VER:      ", client.DRIVER_ODBC_VER)
print ("ODBC_VER:             ", client.ODBC_VER)
print ("ODBC_SQL_CONFORMANCE: ", client.ODBC_SQL_CONFORMANCE)
print ("APPL_CODEPAGE:        ", client.APPL_CODEPAGE)
print ("CONN_CODEPAGE:        ", client.CONN_CODEPAGE)

DRIVER_NAME:           libdb2.a
DRIVER_VER:            11.05.0600
DATA_SOURCE_NAME:      BLUDB
DRIVER_ODBC_VER:       03.51
ODBC_VER:              03.01.0000
ODBC_SQL_CONFORMANCE:  EXTENDED
APPL_CODEPAGE:         1208
CONN_CODEPAGE:         1208


#### Creating table in the database ####

In [13]:
#Lets first drop the table INSTRUCTOR in case it exists from a previous attempt
dropQuery = "drop table INSTRUCTOR"

#Now execute the drop statment
dropStmt = ibm_db.exec_immediate(conn, dropQuery)

#creating table
createQuery = "create table INSTRUCTOR(id INTEGER PRIMARY KEY NOT NULL, fname VARCHAR(20), lname VARCHAR (20), city VARCHAR(20), ccode CHAR(2));"

#xecute the statement
createStmt = ibm_db.exec_immediate(conn, createQuery)

#adding first row
insertQuery = "insert into INSTRUCTOR values(1, 'Rav', 'Ahuja', 'TORONTO','CA');"

#execute the insert statement
insertStmt = ibm_db.exec_immediate(conn, insertQuery)

#adding the rest of the data
insertQuery2 = "insert into INSTRUCTOR values (2, 'Raul', 'Chong', 'Markham', 'CA'), (3,'Hima','Vasudevan', 'Chicago', 'US');"

#execute the statement
insertStmt2 = ibm_db.exec_immediate(conn, insertQuery2)



In [19]:
#Query data in the table

selectQuery = "select * from INSTRUCTOR"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

#Fetch the Dictionary (for the first row only)
ibm_db.fetch_both(selectStmt)

{'ID': 1,
 0: 1,
 'FNAME': 'Rav',
 1: 'Rav',
 'LNAME': 'Ahuja',
 2: 'Ahuja',
 'CITY': 'TORONTO',
 3: 'TORONTO',
 'CCODE': 'CA',
 4: 'CA'}

In [20]:
#Fetch the rest of the rows and print the ID and FNAME for those rows
while ibm_db.fetch_row(selectStmt) != False:
   print (" ID:",  ibm_db.result(selectStmt, 0), " FNAME:",  ibm_db.result(selectStmt, "FNAME"))

 ID: 2  FNAME: Raul
 ID: 3  FNAME: Hima


In [22]:
#update Rav's city to Sydney
updateQuery = "update INSTRUCTOR set CITY='Sydney' where FNAME= 'Rav';"
updateStmt = ibm_db.exec_immediate(conn, updateQuery)


#### Retrieve data into Pandas ####

In [25]:
import pandas
import ibm_db_dbi

pconn = ibm_db_dbi.Connection(conn)

#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#retrieve the query results into a pandas dataframe
pdf = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
print(pdf.LNAME[0])

pdf.shape


Ahuja


(3, 5)

In [26]:
#close the connection
ibm_db.close(conn)

True